In [1]:
import pandas as pd
import re 
import nltk as nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import time
import math
import string
import gensim
import pyLDAvis
import seaborn as sns 
import matplotlib.pyplot as plt
import pyLDAvis.gensim
#import pyLDAvis.gensim_models as gensimvis

from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore

#pyLDAvis.enable_notebook()
#nlp = spacy.load("en_core_web_sm")

In [2]:
import multiprocessing 
multiprocessing.cpu_count()

8

In [3]:
import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/parthbansal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/parthbansal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/parthbansal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/parthbansal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
%%time

df = pd.read_parquet('df_clean.parquet', engine='pyarrow')
df.shape

CPU times: user 2.35 s, sys: 2.18 s, total: 4.54 s
Wall time: 4.51 s


(170984, 9)

In [6]:
df.sample(3, random_state = 42)

,date,title,text,year,month,day,dayofweek,clean_title,clean_text
84444,2021-03-30,Cambridge Quantum Computing Pioneers Quantum M...,Cambridge Quantum Computing Pioneers Quantum M...,2021,3,30,1,Cambridge Quantum Computing Pioneers Quantum M...,Cambridge Quantum Computing Pioneers Quantum M...
35622,2022-12-08,Aetina Launches New ASIC-Based Edge AI System ...,\n\nAetina Launches New ASIC-Based Edge AI Sys...,2022,12,8,3,Aetina Launches New ASICBased Edge AI System P...,Aetina Launches New ASICBased Edge AI System P...
140578,2020-12-04,Emotion AI Software Market To Observe Exponent...,\nEmotion AI Software Market To Observe Expon...,2020,12,4,4,Emotion AI Software Market To Observe Exponent...,Emotion AI Software Market To Observe Exponent...


### Tokenization, and further clean-up

In [7]:
lemma = WordNetLemmatizer()

In [8]:
stopwords = set(nltk.corpus.stopwords.words('english'))
exclude = set(string.punctuation)

In [9]:
def tokens(df,column):
    
    new_col = column + "_tokens"
    df[new_col] = df[column].parallel_apply(lambda x: nltk.tokenize.word_tokenize(x))
    
    df[new_col] = df[new_col].parallel_apply(lambda x: [lemma.lemmatize(word) for word in x])
    df[new_col] = df[new_col].parallel_apply(lambda x: [word.lower() for word in x if word.isalpha()])
    df[new_col] = df[new_col].parallel_apply(lambda x: [word for word in x if word not in stopwords])
    df[new_col] = df[new_col].parallel_apply(lambda x: [word for word in x if len(word)>1])
    df[new_col] = df[new_col].parallel_apply(lambda x: list((x)))

In [10]:
%time tokens(df, "clean_title")

CPU times: user 3.03 s, sys: 1.43 s, total: 4.45 s
Wall time: 8.96 s


In [11]:
%time tokens(df, "clean_text")

CPU times: user 2min 59s, sys: 3min 4s, total: 6min 3s
Wall time: 20min 15s


In [12]:
df.head()

,date,title,text,year,month,day,dayofweek,clean_title,clean_text,clean_title_tokens,clean_text_tokens
0,2021-03-18,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,2021,3,18,3,Artificial intelligence improves parking effic...,Artificial intelligence improves parking effic...,"[artificial, intelligence, improves, parking, ...","[artificial, intelligence, improves, parking, ..."
1,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,2020,2,27,3,Children With Autism Saw Their Learning and So...,Children With Autism Saw Their Learning and So...,"[children, autism, saw, learning, social, skil...","[children, autism, saw, learning, social, skil..."
2,2021-03-26,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce...",2021,3,26,4,"Forget ML, AI and Industry 4.0 obsolescence sh...","Forget ML, AI and Industry 4.0 obsolescence sh...","[forget, ml, ai, industry, obsolescence, focus...","[forget, ml, ai, industry, obsolescence, focus..."
3,2021-03-10,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,2021,3,10,2,Strategy Analytics: 71 of Smartphones Sold Glo...,Strategy Analytics: 71 of Smartphones Sold Glo...,"[strategy, analytics, smartphones, sold, globa...","[strategy, analytics, smartphones, sold, globa..."
4,2020-10-20,Olympus to Support Endoscopic AI Diagnosis Edu...,\n\nOlympus to Support Endoscopic AI Diagnosis...,2020,10,20,1,Olympus to Support Endoscopic AI Diagnosis Edu...,Olympus to Support Endoscopic AI Diagnosis Edu...,"[olympus, support, endoscopic, ai, diagnosis, ...","[olympus, support, endoscopic, ai, diagnosis, ..."


In [13]:
df.to_parquet('df_clean_withtokens.parquet')